## Automatic removal of black slices from medical images and masks (NIfTI files)

This code removes black slices from 3D medical image files and their associated masks in NIfTI format. Black slices do not contain any relevant information, so their removal saves memory and speeds up processing.

In [ ]:
import os
import numpy as np
import nibabel as nib # For loading and saving medical images in NIfTI format.
import logging
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# auxiliary functions

def load_nifti(file_path):
    """
    Loads and returns a NIfTI file.
    """
    try:
        return nib.load(file_path)
    except Exception as e:
        logging.error(f"Error loading file {file_path}: {e}")
        return None

def save_nifti(data, affine, file_path):
    """
    Saves data as NIfTI file.
    """
    try:
        nib.save(nib.Nifti1Image(data, affine), file_path)
    except Exception as e:
        logging.error(f"Error saving file {file_path}: {e}")

# slice filter function

def remove_black_slices(image_path, mask_path, tolerance=1e-6):
    """
    Removes black slices from an image and its associated mask.
    """
    image_nifti = load_nifti(image_path)
    mask_nifti = load_nifti(mask_path)
    
    if image_nifti is None or mask_nifti is None:
        return None, None

    image_data = image_nifti.get_fdata()
    mask_data = mask_nifti.get_fdata()

    # Boolean mask: True for non-black slices
    slice_mask = np.any(mask_data > tolerance, axis=(0, 1))

    # Filtered data based on the mask
    filtered_image_data = image_data[..., slice_mask]
    filtered_mask_data = mask_data[..., slice_mask]

    return filtered_image_data, filtered_mask_data

# Processing of all files with progress indicator

def process_single_file(image_path, mask_path, output_image_path, output_mask_path, tolerance):
    """
    Processes a single image and stores the filtered data.
    """
    filtered_image_data, filtered_mask_data = remove_black_slices(image_path, mask_path, tolerance)

    if filtered_image_data is not None and filtered_mask_data is not None:
        save_nifti(filtered_image_data, nib.load(image_path).affine, output_image_path)
        save_nifti(filtered_mask_data, nib.load(mask_path).affine, output_mask_path)


def process_images_and_masks_parallel(images_dir, masks_dir, output_images_dir, output_masks_dir, tolerance=1e-6):
    """
    Removes black slices from all images and masks in multiple threads.
    """
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_masks_dir, exist_ok=True)

    mask_files = os.listdir(masks_dir)

    with ThreadPoolExecutor() as executor, tqdm(total=len(mask_files)) as progress:
        futures = []
        for mask_file in mask_files:
            mask_path = os.path.join(masks_dir, mask_file)
            image_path = os.path.join(images_dir, mask_file)  # Assumption: same names
            
            if not os.path.exists(image_path):
                logging.warning(f"No corresponding image for mask {mask_file} found.")
                progress.update(1)
                continue

            output_image_path = os.path.join(output_images_dir, mask_file)
            output_mask_path = os.path.join(output_masks_dir, mask_file)

            futures.append(executor.submit(
                process_single_file, image_path, mask_path, output_image_path, output_mask_path, tolerance
            ))

        for future in futures:
            future.result()  # Waiting for the task to complete
            progress.update(1)

# main function

def main():
    images_dir = "data/Spleen/images"
    masks_dir = "data/Spleen/masks"
    output_images_dir = "data/Spleen/output_images"
    output_masks_dir = "data/Spleen/output_masks"

    process_images_and_masks_parallel(images_dir, masks_dir, output_images_dir, output_masks_dir)

if __name__ == "__main__":
    main()


## Segmentation of medical images with MedSAM model and resumability

* ### Importing the necessary libraries
    * #### The code imports basic libraries for scientific computing (numpy), medical imaging (nibabel), machine learning (torch), visualization (matplotlib), and model integration (segment_anything).
    * #### (tqdm) is used for progress indicators, and logging provides structured logging.

* ### Visualization functions
    * #### show_mask: Visualizes a segmentation mask with transparent color coverage.
    * #### show_box: Draws a bounding box that frames the area of ​​the mask.
 
* ### Inference with MedSAM
    * #### medsam_inference:
        * ##### Uses the MedSAM model to create a segmentation for a medical image.
        * ##### Takes an image embedding and a bounding box as input and returns a binary segmentation mask.
        * ##### The results are scaled and returned as a NumPy array.

* ### Device definition
    * ####  The system automatically detects if a GPU is available (torch.cuda.is_available()) and selects it for compute-intensive tasks. Otherwise, the CPU is used.

* ### Loading the MedSAM model
    * #### A pre-trained MedSAM model (vit_b) is loaded, moved to the previously defined device and put into evaluation mode (eval).
    * #### The path to the model checkpoint (sam_vit_b_01ec64.pth) is specified.

* ### Directory structure
    * #### images_dir: Contains the medical images (NIfTI files).
    * #### masks_dir: Contains the associated masks.
    * #### Stores the segmentation results (visualizations and created masks).

* ### Total number of slices
    * #### calculate_total_slices: Calculates the number of slices in all NIfTI images in the directory.
 
* ### Segmentation with continuation
    * #### process_images_and_masks_with_resume:
        * ##### File processing: If a file is missing, a warning is issued.
        * ##### Slice processing: Each slice of an image is processed individually.
        * ##### Bounding box detection: A bounding box is calculated based on non-empty pixels of the mask.
        * ##### Image preprocessing: Images are scaled to 1024x1024 and normalized to meet the requirements of the MedSAM model.
        * ##### Segmentation inference: The MedSAM model creates the segmentation for each slice.
        * ##### Result storage: Visualizations (with bounding box and mask) and the binary segmentation mask are saved in the output directory.
        * ##### Memory cleaning: Memory-intensive objects are deleted and the garbage collector is called.
* ### Progress bar:
    * #### tqdm shows the progress of processing

* ### Main program
    * #### At the end, the function process_images_and_masks_with_resume is called to start the segmentation.
    * #### Optionally, processing can be continued from a specific file and a specific slice.

In [ ]:
# pip install torch torchvision torchaudio

In [ ]:
!where python


In [ ]:
!python --version

In [ ]:
# !pip install torch torchvision

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import transform
import torch
import torch.nn.functional as F
from segment_anything import sam_model_registry
import gc  # for memory cleanup
from tqdm import tqdm
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# %% visualization functions
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251 / 255, 252 / 255, 30 / 255, 0.6])  # yellow with transparency
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor="blue", facecolor=(0, 0, 0, 0), lw=2))

@torch.no_grad()
def medsam_inference(medsam_model, img_embed, box_1024, H, W):
    box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
    if len(box_torch.shape) == 2:
        box_torch = box_torch[:, None, :]  # (B, 1, 4)

    sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
        points=None,
        boxes=box_torch,
        masks=None,
    )
    low_res_logits, _ = medsam_model.mask_decoder(
        image_embeddings=img_embed,
        image_pe=medsam_model.prompt_encoder.get_dense_pe(),
        sparse_prompt_embeddings=sparse_embeddings,
        dense_prompt_embeddings=dense_embeddings,
        multimask_output=False,
    )

    low_res_pred = torch.sigmoid(low_res_logits)  # (1, 1, 256, 256)
    low_res_pred = F.interpolate(
        low_res_pred, size=(H, W), mode="bilinear", align_corners=False
    )  # (1, 1, H, W)
    low_res_pred = low_res_pred.squeeze().cpu().numpy()  # (H, W)
    medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
    return medsam_seg

# %% Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Verwende Gerät: {device}")

# %% Modell laden
MedSAM_CKPT_PATH = "Checkpoint/sam_vit_b_01ec64.pth"  # path to the checkpoint
medsam_model = sam_model_registry["vit_b"](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()
logging.info("MedSAM model loaded successfully.")

# %% Verzeichnisse
# images_dir = "data/Lung Tumors/output_images"
# masks_dir = "data/Lung Tumors/output_masks"
# output_dir = "data/Lung Tumors/segmentation_results"
# os.makedirs(output_dir, exist_ok=True)

# %% Calculate total number of slices
def calculate_total_slices(images_dir):
    total_slices = 0
    for image_file in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_file)
        if not image_file.endswith(".nii") and not image_file.endswith(".nii.gz"):
            continue
        nifti_image = nib.load(image_path)
        total_slices += nifti_image.shape[2]  # Z-Achse
    return total_slices

# %% Optimized segmentation with progress display and logging
def process_images_and_masks_with_resume(images_dir, masks_dir, output_dir, resume_file=None, resume_slice=None):
    """
    Continues segmentation starting from a specific file and slice.
    :param images_dir: Directory containing medical images.
    :param masks_dir: Directory containing masks.
    :param output_dir: Directory for the results.
    :param resume_file: Name of the file to continue from (e.g. 'lung_016.nii.gz').
    :param resume_slice: Slice index to continue from in the file.
    """
    resume = resume_file is None  # If no resume file is specified, start from the beginning

    mask_files = sorted(os.listdir(masks_dir))  # Ensure alphabetical order

    with tqdm(total=len(mask_files), desc="Dateien", unit="Datei") as progress:
        for mask_file in mask_files:
            mask_path = os.path.join(masks_dir, mask_file)
            image_path = os.path.join(images_dir, mask_file)

            if not os.path.exists(image_path):
                logging.warning(f"Fehlendes Bild für Maske: {mask_file}")
                progress.update(1)
                continue

            # When we reach the resume file, activate the resume mode
            if not resume and mask_file == resume_file:
                resume = True

            if not resume:  # Skip files before the resume file
                progress.update(1)
                continue

            # Upload medical image and mask
            nifti_image = nib.load(image_path)
            nifti_mask = nib.load(mask_path)

            image_data = nifti_image.get_fdata()
            mask_data = nifti_mask.get_fdata()

            total_slices = image_data.shape[2]

            # Set the start slice for the resume file
            slice_start = resume_slice if mask_file == resume_file and resume_slice is not None else 0

            with tqdm(total=total_slices, desc=f"Slices in {mask_file}", unit="Slice") as slice_progress:
                for slice_idx in range(slice_start, total_slices):
                    logging.info(f"Process Slice {slice_idx} for file {mask_file}")
                    image_slice = image_data[:, :, slice_idx]
                    mask_slice = mask_data[:, :, slice_idx]

                    if np.sum(mask_slice) == 0:  # Skip empty masks
                        slice_progress.update(1)
                        continue

                    # Calculate Bounding Box from Mask
                    non_zero_coords = np.argwhere(mask_slice > 0)
                    y_min, x_min = non_zero_coords.min(axis=0)
                    y_max, x_max = non_zero_coords.max(axis=0)
                    box_np = np.array([[x_min, y_min, x_max, y_max]])

                    # preprocessing of the image
                    H, W = image_slice.shape
                    resized_image = transform.resize(
                        image_slice, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True
                    )
                    normalized_image = (resized_image - resized_image.min()) / max(resized_image.max() - resized_image.min(), 1e-8)
                    # normalized_image = resized_image / 255.0

                    image_rgb = np.stack([normalized_image] * 3, axis=-1)

                    box_1024 = box_np / np.array([W, H, W, H]) * 1024
                    image_tensor = torch.tensor(image_rgb).float().permute(2, 0, 1).unsqueeze(0).to(device)

                    # MedSAM inference
                    with torch.no_grad():
                        image_embedding = medsam_model.image_encoder(image_tensor)
                        medsam_seg = medsam_inference(medsam_model, image_embedding, box_1024, 1024, 1024)

                    # Save results
                    visualization_file = os.path.join(output_dir, f"{mask_file}_slice_{slice_idx}_visualization.png")
                    seg_file = os.path.join(output_dir, f"{mask_file}_slice_{slice_idx}_segmentation.png")

                    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                    ax[0].imshow(resized_image, cmap="gray")
                    show_box(box_1024[0], ax[0])
                    ax[0].set_title(f"Original Slice {slice_idx} mit Bounding Box")
                    ax[1].imshow(resized_image, cmap="gray")
                    show_mask(medsam_seg, ax[1])
                    ax[1].set_title(f"Segmentierung Slice {slice_idx}")
                    plt.savefig(visualization_file)
                    plt.close(fig)

                    plt.imsave(seg_file, medsam_seg, cmap="gray")

                    # clean memory
                    del image_tensor, image_embedding, medsam_seg
                    gc.collect()

                    slice_progress.update(1)

            logging.info(f"Done with file {mask_file}")
            progress.update(1)

# Define directories
images_dir = "data/Spleen/output_images"
masks_dir = "data/Spleen/output_masks"
output_dir = "data/Spleen/segmentation_results"
os.makedirs(output_dir, exist_ok=True)

# continue processing
process_images_and_masks_with_resume(
    images_dir=images_dir,
    masks_dir=masks_dir,
    output_dir=output_dir,
    resume_file="spleen_2.nii.gz",  # File from which to continue
    resume_slice=0             # Slice from which to continue in the file
)

In [1]:
# -*- coding: utf-8 -*-
import os
import json
import time
import csv
import gc
import logging
from typing import Optional, Tuple

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import transform
import torch
import torch.nn.functional as F
from segment_anything import sam_model_registry
from tqdm import tqdm

# ========================= Logging =========================
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =================== Visualisierungsfunktionen ===================
def show_mask(mask, ax, random_color=False):
    import numpy as np
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251 / 255, 252 / 255, 30 / 255, 0.6])  # Gelb mit Transparenz
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor="blue", facecolor=(0, 0, 0, 0), lw=2))

# ================== MedSAM Inferenz ==================
@torch.no_grad()
def medsam_inference(medsam_model, img_embed, box_1024, H, W):
    box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
    if len(box_torch.shape) == 2:
        box_torch = box_torch[:, None, :]  # (B, 1, 4)

    sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
        points=None, boxes=box_torch, masks=None
    )
    low_res_logits, _ = medsam_model.mask_decoder(
        image_embeddings=img_embed,
        image_pe=medsam_model.prompt_encoder.get_dense_pe(),
        sparse_prompt_embeddings=sparse_embeddings,
        dense_prompt_embeddings=dense_embeddings,
        multimask_output=False,
    )

    low_res_pred = torch.sigmoid(low_res_logits)
    low_res_pred = torch.nn.functional.interpolate(
        low_res_pred, size=(H, W), mode="bilinear", align_corners=False
    )
    low_res_pred = low_res_pred.squeeze().detach().cpu().numpy()
    medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
    return medsam_seg

# =================== Gerät & Modell ===================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Verwende Gerät: {device}")

MedSAM_CKPT_PATH = "Checkpoint/sam_vit_b_01ec64.pth"
medsam_model = sam_model_registry["vit_b"](checkpoint=MedSAM_CKPT_PATH).to(device).eval()
logging.info("MedSAM Modell erfolgreich geladen.")

# =================== Helpers ===================
def _fmt(sec: float) -> str:
    h, r = divmod(sec, 3600)
    m, s = divmod(r, 60)
    return f"{int(h):02d}:{int(m):02d}:{s:05.2f}"

def _sum_seconds_from_csv(csv_path: str) -> float:
    """
    Summiert die Spalte 'seconds' aus timing_per_slice.csv über alle Runs.
    Robuste Auswertung ohne pandas.
    """
    if not os.path.exists(csv_path):
        return 0.0
    total = 0.0
    try:
        with open(csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            if not reader.fieldnames or "seconds" not in reader.fieldnames:
                return 0.0
            for row in reader:
                try:
                    total += float(row["seconds"])
                except Exception:
                    continue
    except Exception as e:
        logging.warning(f"Konnte CSV nicht auswerten ({csv_path}): {e}")
    return total

# =================== Zeitmessung (Wall-Clock + CSV-Gesamt) ===================
def _load_runtime_state(state_path: str) -> dict:
    if os.path.exists(state_path):
        try:
            with open(state_path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            logging.warning(f"Konnte Runtime-State nicht lesen: {e}")
    return {
        "sessions": [],             # [{start, end, elapsed_sec}]
        "total_elapsed_sec": 0.0,   # akkumulierte Wall-Clock-Zeit über Sessions (informativ)
    }

def _save_runtime_state(state_path: str, state: dict):
    with open(state_path, "w", encoding="utf-8") as f:
        json.dump(state, f, indent=2)


def time_run_accumulate(func, state_path: str, seed_timing_csv: str, *args, **kwargs):
    """
    Führt func aus, misst Laufzeit des aktuellen Runs und akkumuliert in state_path.
    Zusätzlich wird am Ende die *echte* Gesamtlaufzeit über alle Neustarts
    aus timing_per_slice.csv (seed_timing_csv) berechnet und geloggt.
    """
    state = _load_runtime_state(state_path)

    start = time.perf_counter()
    start_wall = time.time()
    logging.info("==== Starte Laufzeitmessung (akkumuliert) ====")

    try:
        return func(*args, **kwargs)  # kwargs enthält später timing_csv für die Pipeline
    finally:
        elapsed = time.perf_counter() - start
        end_wall = time.time()

        state["sessions"].append({"start": start_wall, "end": end_wall, "elapsed_sec": elapsed})
        state["total_elapsed_sec"] = float(state.get("total_elapsed_sec", 0.0)) + float(elapsed)
        _save_runtime_state(state_path, state)

        # 1) Wall-Clock (informativ)
        msg1 = f"Aktueller Run: {_fmt(elapsed)} | Akkumuliert (Wall-Clock): {_fmt(state['total_elapsed_sec'])}"
        logging.info(msg1)

        # 2) CSV-basierte Gesamtsumme (entscheidend) – aus seed_timing_csv
        total_csv_sec = _sum_seconds_from_csv(seed_timing_csv)
        msg2 = f"Akkumulierte Gesamtlaufzeit (aus CSV, alle Neustarts): {_fmt(total_csv_sec)}"
        logging.info(msg2)

        with open("runtime_log.txt", "a", encoding="utf-8") as f:
            f.write(msg1 + "\n")
            f.write(msg2 + "\n")

# =================== Resume-Checkpoint ===================
def save_resume_state(checkpoint_path: str, filename: str, slice_idx: int):
    state = {"resume_file": filename, "resume_slice": slice_idx}
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(state, f)

def load_resume_state(checkpoint_path: str) -> Tuple[Optional[str], Optional[int]]:
    if os.path.exists(checkpoint_path):
        try:
            with open(checkpoint_path, "r", encoding="utf-8") as f:
                state = json.load(f)
            return state.get("resume_file"), state.get("resume_slice")
        except Exception as e:
            logging.warning(f"Checkpoint konnte nicht gelesen werden: {e}")
    return None, None

# =================== Pipeline (Spleen) ===================
def process_images_and_masks_with_resume(
    images_dir: str,
    masks_dir: str,
    output_dir: str,
    checkpoint_path: str,
    timing_csv: str,
    resume_file: Optional[str] = None,
    resume_slice: Optional[int] = None,
    use_auto_resume: bool = True,
):
    """
    - Segmentiert alle Spleen-Volumes sliceweise mit MedSAM.
    - Schreibt VOR JEDEM Slice einen Resume-Checkpoint.
    - Hängt pro Slice die Laufzeit an timing_csv an (persistiert über Neustarts).
    """
    os.makedirs(output_dir, exist_ok=True)

    # Auto-Resume
    if use_auto_resume and (resume_file is None and resume_slice is None):
        cf, cs = load_resume_state(checkpoint_path)
        if cf is not None:
            logging.info(f"Automatischer Resume → Datei={cf}, Slice={cs}")
            resume_file, resume_slice = cf, cs

    # timing CSV initialisieren (Header nur einmal)
    if not os.path.exists(timing_csv):
        with open(timing_csv, "w", newline="", encoding="utf-8") as fcsv:
            csv.writer(fcsv).writerow(["file", "slice_idx", "seconds"])

    # Dateiliste
    nii_files = sorted([f for f in os.listdir(images_dir) if f.endswith((".nii", ".nii.gz"))])
    resume_armed = resume_file is None  # erst ab resume_file loslegen

    with tqdm(total=len(nii_files), desc="Dateien", unit="Datei") as prog_files:
        for image_file in nii_files:
            image_path = os.path.join(images_dir, image_file)
            mask_path  = os.path.join(masks_dir,  image_file)  # gleicher Dateiname erwartet

            if not os.path.exists(mask_path):
                logging.warning(f"Fehlende Maske: {image_file}")
                prog_files.update(1)
                continue

            if not resume_armed:
                if image_file == resume_file:
                    resume_armed = True
                else:
                    prog_files.update(1)
                    continue

            # NIfTI laden
            nifti_image = nib.load(image_path)
            nifti_mask  = nib.load(mask_path)
            img = nifti_image.get_fdata()
            msk = nifti_mask.get_fdata()

            total_slices = img.shape[2]
            slice_start = resume_slice if (image_file == resume_file and resume_slice is not None) else 0

            with tqdm(total=total_slices, desc=f"Slices in {image_file}", unit="Slice") as prog_slices:
                for slice_idx in range(slice_start, total_slices):
                    t0 = time.perf_counter()
                    try:
                        # Checkpoint vor der Bearbeitung schreiben
                        save_resume_state(checkpoint_path, image_file, slice_idx)

                        image_slice = img[:, :, slice_idx]
                        mask_slice  = msk[:, :, slice_idx]

                        if np.sum(mask_slice) == 0:
                            # leere GT → Überspringen (aber Zeit/Progress zählen)
                            continue

                        # Vorverarbeitung
                        H, W = image_slice.shape
                        resized_image = transform.resize(
                            image_slice, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True
                        )
                        normalized_image = (resized_image - resized_image.min()) / max(resized_image.ptp(), 1e-8)
                        image_rgb = np.stack([normalized_image] * 3, axis=-1)
                        image_tensor = torch.tensor(image_rgb).float().permute(2, 0, 1).unsqueeze(0).to(device)

                        # Bounding Box aus Maske
                        yx = np.argwhere(mask_slice > 0)
                        y_min, x_min = yx.min(axis=0)
                        y_max, x_max = yx.max(axis=0)
                        box_np = np.array([[x_min, y_min, x_max, y_max]])
                        box_1024 = box_np / np.array([W, H, W, H]) * 1024

                        # MedSAM
                        with torch.no_grad():
                            image_embedding = medsam_model.image_encoder(image_tensor)
                            medsam_seg = medsam_inference(medsam_model, image_embedding, box_1024, 1024, 1024)

                        # Speichern
                        vis_file = os.path.join(output_dir, f"{image_file}_slice_{slice_idx}_visualization.png")
                        seg_file = os.path.join(output_dir, f"{image_file}_slice_{slice_idx}_segmentation.png")

                        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                        ax[0].imshow(resized_image, cmap="gray")
                        show_box(box_1024[0], ax[0])
                        ax[0].set_title(f"Original Slice {slice_idx} mit Bounding Box")
                        ax[1].imshow(resized_image, cmap="gray")
                        show_mask(medsam_seg, ax[1])
                        ax[1].set_title(f"Segmentierung Slice {slice_idx}")
                        plt.savefig(vis_file)
                        plt.close(fig)

                        plt.imsave(seg_file, medsam_seg, cmap="gray")

                        # Cleanup
                        del image_tensor, image_embedding, medsam_seg
                        gc.collect()

                    except Exception as e:
                        logging.error(f"Fehler bei {image_file} Slice {slice_idx}: {e}")
                    finally:
                        # Slice-Zeit anhängen (persistiert über alle Runs)
                        elapsed_slice = time.perf_counter() - t0
                        with open(timing_csv, "a", newline="", encoding="utf-8") as fcsv:
                            csv.writer(fcsv).writerow([image_file, slice_idx, f"{elapsed_slice:.4f}"])
                        prog_slices.update(1)

            prog_files.update(1)

    # Fertig → Checkpoint löschen
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        logging.info("Checkpoint entfernt (Pipeline abgeschlossen).")

# =================== AUSFÜHRUNG (Spleen) ===================
if __name__ == "__main__":
    images_dir = os.path.join("data", "Spleen", "output_images")
    masks_dir  = os.path.join("data", "Spleen", "output_masks")
    output_dir = os.path.join("data", "Spleen", "segmentation_results_with_run_time")
    os.makedirs(output_dir, exist_ok=True)

    checkpoint_path = os.path.join(output_dir, "resume_checkpoint.json")
    timing_csv      = os.path.join(output_dir, "timing_per_slice.csv")
    runtime_state   = os.path.join(output_dir, "runtime_state.json")

    # Manuelles Resume (optional). Wenn None/None → Auto-Resume via JSON.
    manual_resume_file  = "spleen_63.nii.gz"   # z.B. "spleen_44.nii.gz"
    manual_resume_slice = 20   # z.B. 0

    # Lauf starten (Wall-Clock akkumuliert + CSV-basierte Gesamtzeit)

    time_run_accumulate(
        process_images_and_masks_with_resume,
        runtime_state,              # state_path
        seed_timing_csv=timing_csv, # <-- NUR für den Wrapper (Summe am Ende)
        images_dir=images_dir,
        masks_dir=masks_dir,
        output_dir=output_dir,
        checkpoint_path=checkpoint_path,
        timing_csv=timing_csv,      # <-- für die Pipeline-Funktion (Pflichtargument)
        resume_file=manual_resume_file,
        resume_slice=manual_resume_slice,
        use_auto_resume=True,
    )

    # Optional: Hier könntest du noch einmal explizit die CSV-Gesamtzeit loggen:
    total_csv_sec = _sum_seconds_from_csv(timing_csv)
    logging.info(f"[FINAL] Gesamtlaufzeit aus CSV (alle Neustarts): {_fmt(total_csv_sec)}")


2025-09-06 23:37:45,973 - INFO - Verwende Gerät: cpu
2025-09-06 23:37:56,580 - INFO - MedSAM Modell erfolgreich geladen.
2025-09-06 23:37:56,602 - INFO - ==== Starte Laufzeitmessung (akkumuliert) ====
Dateien: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [25:05<00:00, 36.71s/Datei]
2025-09-07 00:03:01,657 - INFO - Checkpoint entfernt (Pipeline abgeschlossen).
2025-09-07 00:03:01,676 - INFO - Aktueller Run: 00:25:05.07 | Akkumuliert (Wall-Clock): 00:25:05.07
2025-09-07 00:03:01,739 - INFO - Akkumulierte Gesamtlaufzeit (aus CSV, alle Neustarts): 10:55:52.90
2025-09-07 00:03:01,768 - INFO - [FINAL] Gesamtlaufzeit aus CSV (alle Neustarts): 10:55:52.90


In [2]:
import pandas as pd

# Pfad zur timing_per_slice.csv
csv_path = "data/Spleen/segmentation_results_with_run_time/timing_per_slice.csv"

# CSV laden
df = pd.read_csv(csv_path)

# Spalte seconds sicher als float laden
df['seconds'] = pd.to_numeric(df['seconds'], errors='coerce')

# Gesamtzeit
total_seconds = df['seconds'].sum()
total_minutes = total_seconds / 60
total_hours = total_seconds / 3600

# Anzahl der Slices
num_slices = len(df)

# Durchschnittszeit
avg_sec = df['seconds'].mean()
avg_min = avg_sec / 60

# Min/Max/Median
min_sec = df['seconds'].min()
max_sec = df['seconds'].max()
median_sec = df['seconds'].median()

# Ausgabe auf der Konsole
print("==== Timing Summary ====")
print(f"Slices verarbeitet : {num_slices}")
print(f"Gesamtzeit        : {total_seconds:.2f} Sekunden")
print(f"                   : {total_minutes:.2f} Minuten")
print(f"                   : {total_hours:.2f} Stunden")
print(f"Durchschnitt/Slice: {avg_sec:.2f} Sekunden ({avg_min:.2f} Minuten)")
print(f"Median/Slice      : {median_sec:.2f} Sekunden")
print(f"Schnellster Slice : {min_sec:.2f} Sekunden")
print(f"Langsamster Slice : {max_sec:.2f} Sekunden")

# Optional: Zusammenfassung in eine Textdatei speichern
summary_path = "data/Spleen/segmentation_results/timing_summary.txt"
with open(summary_path, "w") as f:
    f.write("==== Timing Summary ====\n")
    f.write(f"Slices verarbeitet : {num_slices}\n")
    f.write(f"Gesamtzeit        : {total_seconds:.2f} Sekunden\n")
    f.write(f"                   : {total_minutes:.2f} Minuten\n")
    f.write(f"                   : {total_hours:.2f} Stunden\n")
    f.write(f"Durchschnitt/Slice: {avg_sec:.2f} Sekunden ({avg_min:.2f} Minuten)\n")
    f.write(f"Median/Slice      : {median_sec:.2f} Sekunden\n")
    f.write(f"Schnellster Slice : {min_sec:.2f} Sekunden\n")
    f.write(f"Langsamster Slice : {max_sec:.2f} Sekunden\n")

print(f"\nZusammenfassung gespeichert in {summary_path}")


==== Timing Summary ====
Slices verarbeitet : 1060
Gesamtzeit        : 39352.90 Sekunden
                   : 655.88 Minuten
                   : 10.93 Stunden
Durchschnitt/Slice: 37.13 Sekunden (0.62 Minuten)
Median/Slice      : 30.19 Sekunden
Schnellster Slice : 24.31 Sekunden
Langsamster Slice : 5370.33 Sekunden

Zusammenfassung gespeichert in data/Spleen/segmentation_results/timing_summary.txt


## Conversion and adaptation of mask slices from NIfTI files to PNG format for the calculation of Dice Score later

This code converts masks from medical image files (NIfTI format) to PNG image format.
The mask slices are converted to binary masks, resized and saved to a destination folder.

In [ ]:
import os
import cv2
import nibabel as nib
import numpy as np
from PIL import Image
import logging

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Eingabe- und Ausgabeordner
input_mask_folder = os.path.join("data", "Spleen", "output_masks")
updated_mask_folder = os.path.join("data", "Spleen", "output_masks_updated")
os.makedirs(updated_mask_folder, exist_ok=True)

# Zielauflösung
target_resolution = (1024, 1024)

# Verarbeitung der Masken
for mask_filename in os.listdir(input_mask_folder):
    if mask_filename.endswith((".nii", ".nii.gz")):
        mask_path = os.path.join(input_mask_folder, mask_filename)
        try:
            mask_nii = nib.load(mask_path)
        except Exception as e:
            logging.error(f"Fehler beim Laden von {mask_filename}: {e}")
            continue

        mask_data = mask_nii.get_fdata()
        base_name = mask_filename.split(".nii")[0]

        for slice_idx in range(mask_data.shape[2]):
            slice_data = mask_data[:, :, slice_idx]
            slice_data = (slice_data > 0).astype(np.uint8) * 255  # Binärmaske

            # Slice-Nummer formatieren und Skalierung durchführen
            slice_number_formatted = f"{slice_idx + 1:02d}"
            updated_filename = f"{base_name}_slice_{slice_number_formatted}_segmentation.png"

            slice_image = Image.fromarray(slice_data)
            slice_image_resized = slice_image.resize(target_resolution, Image.LANCZOS)

            updated_path = os.path.join(updated_mask_folder, updated_filename)
            try:
                slice_image_resized.save(updated_path)
                logging.info(f"Gespeichert: {updated_filename}")
            except Exception as e:
                logging.error(f"Fehler beim Speichern von {updated_filename}: {e}")

logging.info(f"Alle Masken wurden im Ordner '{updated_mask_folder}' gespeichert.")


## Calculating the Dice Similarity Coefficient (DSC)

In [ ]:
import os
import numpy as np
import cv2
import logging
import csv
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordner mit den Masken
mask_png_folder = os.path.join("data", "Spleen", "output_masks_updated")
segmentation_results_folder = os.path.join("data", "Spleen", "segmentation_results_with_run_time")

# Funktion zur Berechnung des Dice Scores
def dice_score(mask1, mask2):
    intersection = np.sum(mask1 * mask2)
    if np.sum(mask1) + np.sum(mask2) == 0:
        return 1.0  # Sonderfall: Beide Masken sind komplett leer
    dice = (2. * intersection) / (np.sum(mask1) + np.sum(mask2))
    return dice

# Funktion zur Verarbeitung einer einzelnen Datei
def process_single_file(mask_filename):
    base_name = mask_filename.split("_slice_")[0]
    slice_number = mask_filename.split("_slice_")[-1].split("_")[0]
    slice_number_converted = int(slice_number) - 1
    segmentation_filename = f"{base_name}.nii.gz_slice_{slice_number_converted}_segmentation.png"

    mask_png_path = os.path.join(mask_png_folder, mask_filename)
    segmentation_path = os.path.join(segmentation_results_folder, segmentation_filename)

    if os.path.exists(segmentation_path):
        try:
            # Masken laden
            mask_png = cv2.imread(mask_png_path, cv2.IMREAD_GRAYSCALE)
            segmentation = cv2.imread(segmentation_path, cv2.IMREAD_GRAYSCALE)

            # Sicherstellen, dass beide Masken binär sind (0 und 1)
            mask_png = (mask_png > 0).astype(np.uint8)
            segmentation = (segmentation > 0).astype(np.uint8)

            # Dice Score berechnen
            dice = dice_score(mask_png, segmentation)
            logging.info(f"{mask_filename}: Dice Score = {dice:.4f}")
            return mask_filename, dice
        except Exception as e:
            logging.error(f"Fehler beim Verarbeiten von {mask_filename}: {e}")
            return None
    else:
        logging.warning(f"Segmentationsdatei für {mask_filename} nicht gefunden!")
        return None

# Parallelisierte Verarbeitung
dice_scores = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_single_file, mask_filename) for mask_filename in os.listdir(mask_png_folder) if mask_filename.endswith(".png")]
    dice_scores = [future.result() for future in futures if future.result() is not None]


2025-09-08 11:43:24,310 - INFO - spleen_3_slice_08_segmentation.png: Dice Score = 0.9638
2025-09-08 11:43:24,321 - INFO - spleen_3_slice_06_segmentation.png: Dice Score = 0.9390
2025-09-08 11:43:24,340 - INFO - spleen_3_slice_05_segmentation.png: Dice Score = 0.9352
2025-09-08 11:43:24,386 - INFO - spleen_3_slice_01_segmentation.png: Dice Score = 0.5340
2025-09-08 11:43:24,393 - INFO - spleen_3_slice_09_segmentation.png: Dice Score = 0.9645
2025-09-08 11:43:24,400 - INFO - spleen_3_slice_02_segmentation.png: Dice Score = 0.9041
2025-09-08 11:43:24,393 - INFO - spleen_3_slice_07_segmentation.png: Dice Score = 0.8873
2025-09-08 11:43:24,432 - INFO - spleen_3_slice_03_segmentation.png: Dice Score = 0.9081
2025-09-08 11:43:24,447 - INFO - spleen_3_slice_12_segmentation.png: Dice Score = 0.9442
2025-09-08 11:43:24,452 - INFO - spleen_3_slice_10_segmentation.png: Dice Score = 0.9583
2025-09-08 11:43:24,467 - INFO - spleen_3_slice_04_segmentation.png: Dice Score = 0.9203
2025-09-08 11:43:24,4

In [ ]:
# Ergebnisse speichern und analysieren
if dice_scores:
    # Ergebnisse in CSV speichern
    with open("dice_scores.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Maskenname", "Dice Score"])
        writer.writerows(dice_scores)

    # Durchschnitt berechnen
    dice_values = [score for _, score in dice_scores]
    avg_dice = np.mean(dice_values)
    logging.info(f"\nDurchschnittlicher Dice Score: {avg_dice:.4f}")

    # Histogramm erstellen
    plt.figure(figsize=(8, 6))
    plt.hist(dice_values, bins=20, color='blue', alpha=0.7, edgecolor='black')
    plt.title('Distribution of dice scores')
    plt.xlabel('Dice Score')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Durchschnitt im Histogramm markieren
    plt.axvline(avg_dice, color='red', linestyle='dashed', linewidth=1.5, label=f'Durchschnitt: {avg_dice:.4f}')
    plt.legend()

    # Plot speichern und anzeigen
    plt.savefig('data\Spleen\dice_score_histogram_MedSAM_Spleen.png')
    plt.show()
else:
    logging.info("Keine Dice Scores berechnet!")


In [ ]:
import nibabel as nib

nii = nib.load("data/Spleen/output_images/spleen_63.nii.gz")
spacing = nii.header.get_zooms()[:2]  # (dy, dx) Pixel Spacing in mm
print("Pixel Spacing:", spacing)


In [ ]:
import os
import numpy as np
import cv2
import logging
import csv
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from scipy import ndimage as ndi

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------- Pfade (wie bei dir) ----------------
mask_png_folder = os.path.join("data", "Spleen", "output_masks_updated")
segmentation_results_folder = os.path.join("data", "Spleen", "segmentation_results")

# ---------------- Parameter ----------------
# Pixelabstände in mm (für PNGs meist unbekannt → 1.0 annehmen, sonst hier anpassen)
pixel_spacing = (1.0, 1.0)  # (dy, dx) in mm

# ---------------- Hilfsfunktionen ----------------
def _check_binary_2d(a: np.ndarray) -> np.ndarray:
    """Sicherstellen, dass 2D & binär (0/1)."""
    if a.ndim != 2:
        raise ValueError("Erwarte 2D-Maske (ein Slice).")
    return (a > 0).astype(np.uint8)

def _surface_2d(mask: np.ndarray) -> np.ndarray:
    """2D-Oberflächenpunkte (Randpixel) einer binären Maske."""
    if mask.sum() == 0:
        return mask.astype(bool)
    eroded = ndi.binary_erosion(mask, iterations=1, border_value=0)
    surf = mask.astype(bool) & (~eroded)
    return surf

def _surface_distances_2d(A: np.ndarray, B: np.ndarray, spacing=(1.0, 1.0)) -> np.ndarray:
    """
    Abstände von der Oberfläche von A zur Oberfläche von B (symmetrische Metriken nutzen beide Richtungen).
    Implementierung via EDT des Komplements von B.
    """
    if B.sum() == 0:
        # keine B-Pixel vorhanden -> alle A-Oberflächen haben "unendlichen" Abstand
        d = np.full(A.shape, np.inf, dtype=float)
    else:
        d = ndi.distance_transform_edt(~B.astype(bool), sampling=spacing)
    A_surf = _surface_2d(A)
    return d[A_surf]

def hausdorff_2d(mask1: np.ndarray, mask2: np.ndarray, spacing=(1.0, 1.0), percentile: float | None = None) -> float:
    """
    Hausdorff-Distanz (in mm) zwischen zwei 2D-Binärmasken.
    - percentile=None → klassische HD (Maximum)
    - percentile=95 → HD95 (robuster ggü. Ausreißern)
    """
    a = _check_binary_2d(mask1)
    b = _check_binary_2d(mask2)

    # Sonderfälle
    if a.sum() == 0 and b.sum() == 0:
        return 0.0
    if a.sum() == 0 or b.sum() == 0:
        return float("inf")

    d_ab = _surface_distances_2d(a, b, spacing)
    d_ba = _surface_distances_2d(b, a, spacing)
    all_d = np.concatenate([d_ab, d_ba])

    if all_d.size == 0:
        return 0.0

    if percentile is None:
        return float(np.max(all_d))
    else:
        return float(np.percentile(all_d, percentile))

# ---------------- Einzeldatei-Verarbeitung ----------------
def process_single_file_hd(mask_filename: str):
    """
    Erwartet Masken wie: <basename>_slice_{N}_mask.png
    Sucht dazu:          <basename>.nii.gz_slice_{N-1}_segmentation.png
    """
    base_name = mask_filename.split("_slice_")[0]
    slice_number = mask_filename.split("_slice_")[-1].split("_")[0]
    slice_number_converted = int(slice_number) - 1
    segmentation_filename = f"{base_name}.nii.gz_slice_{slice_number_converted}_segmentation.png"

    mask_png_path = os.path.join(mask_png_folder, mask_filename)
    segmentation_path = os.path.join(segmentation_results_folder, segmentation_filename)

    if not os.path.exists(segmentation_path):
        logging.warning(f"Segmentationsdatei für {mask_filename} nicht gefunden!")
        return None

    try:
        # Masken laden (grau) und binär machen
        mask_png = cv2.imread(mask_png_path, cv2.IMREAD_GRAYSCALE)
        segmentation = cv2.imread(segmentation_path, cv2.IMREAD_GRAYSCALE)

        if mask_png is None or segmentation is None:
            logging.error(f"Fehler beim Laden: {mask_filename}")
            return None

        mask_bin = (mask_png > 0).astype(np.uint8)
        seg_bin  = (segmentation > 0).astype(np.uint8)

        # HD & HD95 berechnen
        hd   = hausdorff_2d(mask_bin, seg_bin, spacing=pixel_spacing, percentile=None)
        hd95 = hausdorff_2d(mask_bin, seg_bin, spacing=pixel_spacing, percentile=95)

        logging.info(f"{mask_filename}: HD = {hd:.3f} mm | HD95 = {hd95:.3f} mm")
        return mask_filename, hd, hd95

    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten von {mask_filename}: {e}")
        return None

# ---------------- Parallelisieren & Sammeln ----------------
results = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_single_file_hd, fn)
        for fn in os.listdir(mask_png_folder)
        if fn.endswith(".png")
    ]
    results = [f.result() for f in futures if f.result() is not None]

# ---------------- Speichern & Visualisieren ----------------
if results:
    # CSV
    csv_path = "MedSAM_Spleen_hausdorff_results.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Maskenname", "Hausdorff_mm", "HD95_mm"])
        writer.writerows(results)
    logging.info(f"CSV gespeichert: {csv_path}")

    # Arrays für Plots
    hd_vals   = [r[1] for r in results if np.isfinite(r[1])]
    hd95_vals = [r[2] for r in results if np.isfinite(r[2])]

    # Mittelwerte (ohne inf)
    if hd_vals:
        avg_hd = np.mean(hd_vals)
        logging.info(f"Durchschnittliche HD (ohne ∞): {avg_hd:.3f} mm")
    else:
        avg_hd = None

    if hd95_vals:
        avg_hd95 = np.mean(hd95_vals)
        logging.info(f"Durchschnittliche HD95: {avg_hd95:.3f} mm")
    else:
        avg_hd95 = None

    # Histogramm HD
    if hd_vals:
        plt.figure(figsize=(8, 6))
        plt.hist(hd_vals, bins=20, color='blue', alpha=0.7, edgecolor='black')
        plt.title('Distribution of Hausdorff Distance (mm)')
        plt.xlabel('HD (mm)')
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        if avg_hd is not None:
            plt.axvline(avg_hd, color='red', linestyle='dashed', linewidth=1.5, label=f'Avg HD: {avg_hd:.3f} mm')
            plt.legend()
        plot_path_hd = os.path.join("data", "Spleen", "hausdorff_histogram_MedSAM_Spleen.png")
        plt.savefig(plot_path_hd, dpi=150, bbox_inches='tight')
        plt.show()
        logging.info(f"HD-Histogramm gespeichert: {plot_path_hd}")

    # Histogramm HD95
    if hd95_vals:
        plt.figure(figsize=(8, 6))
        plt.hist(hd95_vals, bins=20, color='green', alpha=0.7, edgecolor='black')
        plt.title('Distribution of 95th Percentile Hausdorff Distance (HD95, mm)')
        plt.xlabel('HD95 (mm)')
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        if avg_hd95 is not None:
            plt.axvline(avg_hd95, color='red', linestyle='dashed', linewidth=1.5, label=f'Avg HD95: {avg_hd95:.3f} mm')
            plt.legend()
        plot_path_hd95 = os.path.join("data", "Spleen", "hausdorff95_histogram_MedSAM_Spleen.png")
        plt.savefig(plot_path_hd95, dpi=150, bbox_inches='tight')
        plt.show()
        logging.info(f"HD95-Histogramm gespeichert: {plot_path_hd95}")

else:
    logging.info("Keine HD/HD95-Werte berechnet!")


### angepassten Code schreiben, der das Pixel Spacing automatisch aus den NIfTI-Dateien ausliest und pro Slice korrekt anwendet?

In [ ]:
import os
import numpy as np
import cv2
import logging
import csv
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from scipy import ndimage as ndi
import nibabel as nib  # <— NEU

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------- Pfade ----------------
mask_png_folder = os.path.join("data", "Spleen", "output_masks_updated")
segmentation_results_folder = os.path.join("data", "Spleen", "segmentation_results")
nii_root = os.path.join("data", "Spleen", "output_images")  # <— Ordner mit deinen .nii/.nii.gz Dateien

# ---------------- Spacing-Cache ----------------
spacing_cache = {}  # map: base_name -> (dy, dx) in mm

def _get_spacing_from_nifti(base_name: str) -> tuple[float, float]:
    """
    Liest (dy, dx) aus dem NIfTI-Header von <nii_root>/<base_name>.nii.gz bzw. .nii.
    Cacht das Ergebnis. Fallback: (1.0, 1.0) mit Warnung.
    """
    if base_name in spacing_cache:
        return spacing_cache[base_name]

    candidates = [
        os.path.join(nii_root, f"{base_name}.nii.gz"),
        os.path.join(nii_root, f"{base_name}.nii"),
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                nii = nib.load(p)
                zooms = nii.header.get_zooms()
                # NIfTI: (dx, dy, dz, ...) — wir brauchen (dy, dx) für 2D-EDT
                # Viele Pipelines legen (x,y,...) an; unsere Hausdorff-Funktion erwartet (dy, dx):
                if len(zooms) >= 2:
                    dx, dy = float(zooms[0]), float(zooms[1])
                    spacing = (dy, dx)
                    spacing_cache[base_name] = spacing
                    return spacing
            except Exception as e:
                logging.warning(f"Kann NIfTI laden, aber Header nicht lesbar für {base_name}: {e}")

    logging.warning(f"Kein NIfTI für {base_name} gefunden → spacing=(1.0, 1.0) als Fallback.")
    spacing_cache[base_name] = (1.0, 1.0)
    return (1.0, 1.0)

# ---------------- Hilfsfunktionen ----------------
def _check_binary_2d(a: np.ndarray) -> np.ndarray:
    if a.ndim != 2:
        raise ValueError("Erwarte 2D-Maske (ein Slice).")
    return (a > 0).astype(np.uint8)

def _surface_2d(mask: np.ndarray) -> np.ndarray:
    if mask.sum() == 0:
        return mask.astype(bool)
    eroded = ndi.binary_erosion(mask, iterations=1, border_value=0)
    surf = mask.astype(bool) & (~eroded)
    return surf

def _surface_distances_2d(A: np.ndarray, B: np.ndarray, spacing=(1.0, 1.0)) -> np.ndarray:
    if B.sum() == 0:
        d = np.full(A.shape, np.inf, dtype=float)
    else:
        d = ndi.distance_transform_edt(~B.astype(bool), sampling=spacing)
    A_surf = _surface_2d(A)
    return d[A_surf]

def hausdorff_2d(mask1: np.ndarray, mask2: np.ndarray, spacing=(1.0, 1.0), percentile: float | None = None) -> float:
    a = _check_binary_2d(mask1)
    b = _check_binary_2d(mask2)

    if a.sum() == 0 and b.sum() == 0:
        return 0.0
    if a.sum() == 0 or b.sum() == 0:
        return float("inf")

    d_ab = _surface_distances_2d(a, b, spacing)
    d_ba = _surface_distances_2d(b, a, spacing)
    all_d = np.concatenate([d_ab, d_ba])

    if all_d.size == 0:
        return 0.0

    return float(np.max(all_d) if percentile is None else np.percentile(all_d, percentile))

# ---------------- Einzeldatei-Verarbeitung ----------------
def process_single_file_hd(mask_filename: str):
    """
    Erwartet Masken wie: <basename>_slice_{N}_mask.png
    Sucht dazu:          <basename>.nii.gz_slice_{N-1}_segmentation.png
    (lass das so — dein Schema funktioniert bereits)
    """
    base_name = mask_filename.split("_slice_")[0]  # z.B. "spleen_63"
    slice_number = mask_filename.split("_slice_")[-1].split("_")[0]
    slice_number_converted = int(slice_number) - 1
    segmentation_filename = f"{base_name}.nii.gz_slice_{slice_number_converted}_segmentation.png"

    mask_png_path = os.path.join(mask_png_folder, mask_filename)
    segmentation_path = os.path.join(segmentation_results_folder, segmentation_filename)

    if not os.path.exists(segmentation_path):
        logging.warning(f"Segmentationsdatei für {mask_filename} nicht gefunden!")
        return None

    try:
        # 1) Spacing aus NIfTI holen (pro Volume)
        spacing = _get_spacing_from_nifti(base_name)  # (dy, dx) in mm

        # 2) Masken laden
        mask_png = cv2.imread(mask_png_path, cv2.IMREAD_GRAYSCALE)
        segmentation = cv2.imread(segmentation_path, cv2.IMREAD_GRAYSCALE)

        if mask_png is None or segmentation is None:
            logging.error(f"Fehler beim Laden: {mask_filename}")
            return None

        # (Optional) Größenabgleich — wenn deine Pipeline garantiert gleiche Shapes liefert, kann dieser Block weg
        if mask_png.shape != segmentation.shape:
            logging.warning(f"Resize {base_name} slice {slice_number}: {segmentation.shape} -> {mask_png.shape}")
            segmentation = cv2.resize(segmentation, (mask_png.shape[1], mask_png.shape[0]), interpolation=cv2.INTER_NEAREST)

        mask_bin = (mask_png > 0).astype(np.uint8)
        seg_bin  = (segmentation > 0).astype(np.uint8)

        # 3) HD / HD95 mit KORREKTEM spacing
        hd   = hausdorff_2d(mask_bin, seg_bin, spacing=spacing, percentile=None)
        hd95 = hausdorff_2d(mask_bin, seg_bin, spacing=spacing, percentile=95)

        logging.info(f"{mask_filename}: HD = {hd:.3f} mm | HD95 = {hd95:.3f} mm (spacing={spacing})")
        return mask_filename, hd, hd95

    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten von {mask_filename}: {e}")
        return None

# ---------------- Parallelisieren & Sammeln ----------------
results = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_single_file_hd, fn)
        for fn in os.listdir(mask_png_folder)
        if fn.endswith(".png")
    ]
    results = [f.result() for f in futures if f.result() is not None]

# ---------------- Speichern & Visualisieren ----------------
if results:
    csv_path = "MedSAM_Spleen_hausdorff_results.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Maskenname", "Hausdorff_mm", "HD95_mm"])
        writer.writerows(results)
    logging.info(f"CSV gespeichert: {csv_path}")

    hd_vals   = [r[1] for r in results if np.isfinite(r[1])]
    hd95_vals = [r[2] for r in results if np.isfinite(r[2])]

    avg_hd = np.mean(hd_vals) if hd_vals else None
    avg_hd95 = np.mean(hd95_vals) if hd95_vals else None

    if hd_vals:
        plt.figure(figsize=(8, 6))
        plt.hist(hd_vals, bins=20, color='blue', alpha=0.7, edgecolor='black')
        plt.title('Distribution of Hausdorff Distance (mm)')
        plt.xlabel('HD (mm)'); plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        if avg_hd is not None:
            plt.axvline(avg_hd, color='red', linestyle='dashed', linewidth=1.5, label=f'Avg HD: {avg_hd:.3f} mm')
            plt.legend()
        plot_path_hd = os.path.join("data", "Spleen", "hausdorff_histogram_MedSAM_Spleen.png")
        plt.savefig(plot_path_hd, dpi=150, bbox_inches='tight'); plt.show()
        logging.info(f"HD-Histogramm gespeichert: {plot_path_hd}")

    if hd95_vals:
        plt.figure(figsize=(8, 6))
        plt.hist(hd95_vals, bins=20, color='green', alpha=0.7, edgecolor='black')
        plt.title('Distribution of 95th Percentile Hausdorff Distance (HD95, mm)')
        plt.xlabel('HD95 (mm)'); plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        if avg_hd95 is not None:
            plt.axvline(avg_hd95, color='red', linestyle='dashed', linewidth=1.5, label=f'Avg HD95: {avg_hd95:.3f} mm')
            plt.legend()
        plot_path_hd95 = os.path.join("data", "Spleen", "hausdorff95_histogram_MedSAM_Spleen.png")
        plt.savefig(plot_path_hd95, dpi=150, bbox_inches='tight'); plt.show()
        logging.info(f"HD95-Histogramm gespeichert: {plot_path_hd95}")
else:
    logging.info("Keine HD/HD95-Werte berechnet!")


In [ ]:
import os
import numpy as np
import cv2
import logging
import csv
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------- Pfade ----------------
mask_png_folder = os.path.join("data", "Spleen", "output_masks_updated")
segmentation_results_folder = os.path.join("data", "Spleen", "segmentation_results")

# ---------------- IoU ----------------
def iou_2d(mask1: np.ndarray, mask2: np.ndarray) -> float:
    """
    Intersection over Union für 2D-Binärmasken.
    IoU = |A ∩ B| / |A ∪ B|
    Sonderfall: beide leer -> IoU = 1.0
    """
    a = (mask1 > 0).astype(np.uint8)
    b = (mask2 > 0).astype(np.uint8)
    inter = np.logical_and(a, b).sum()
    union = np.logical_or(a, b).sum()
    if union == 0:
        return 1.0
    return inter / union

# ---------------- Einzeldatei-Verarbeitung ----------------
def process_single_file_iou(mask_filename: str):
    """
    Erwartet Masken wie: <basename>_slice_{N}_mask.png
    Sucht dazu:          <basename>.nii.gz_slice_{N-1}_segmentation.png
    (gleiches Schema wie in deinem HD-Code)
    """
    base_name = mask_filename.split("_slice_")[0]
    slice_number = mask_filename.split("_slice_")[-1].split("_")[0]
    slice_number_converted = int(slice_number) - 1
    segmentation_filename = f"{base_name}.nii.gz_slice_{slice_number_converted}_segmentation.png"

    mask_png_path = os.path.join(mask_png_folder, mask_filename)
    segmentation_path = os.path.join(segmentation_results_folder, segmentation_filename)

    if not os.path.exists(segmentation_path):
        logging.warning(f"Segmentationsdatei für {mask_filename} nicht gefunden!")
        return None

    try:
        # Masken laden (grau) und binär machen
        mask_png = cv2.imread(mask_png_path, cv2.IMREAD_GRAYSCALE)
        segmentation = cv2.imread(segmentation_path, cv2.IMREAD_GRAYSCALE)

        if mask_png is None or segmentation is None:
            logging.error(f"Fehler beim Laden: {mask_filename}")
            return None

        # Falls Formen abweichen, Segmentation auf Maske bringen (Nearest)
        if mask_png.shape != segmentation.shape:
            logging.warning(f"Resize {base_name} slice {slice_number}: {segmentation.shape} -> {mask_png.shape}")
            segmentation = cv2.resize(segmentation, (mask_png.shape[1], mask_png.shape[0]), interpolation=cv2.INTER_NEAREST)

        mask_bin = (mask_png > 0).astype(np.uint8)
        seg_bin  = (segmentation > 0).astype(np.uint8)

        # IoU berechnen
        iou_val = iou_2d(mask_bin, seg_bin)
        logging.info(f"{mask_filename}: IoU = {iou_val:.4f}")
        return mask_filename, iou_val

    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten von {mask_filename}: {e}")
        return None

# ---------------- Parallelisieren & Sammeln ----------------
iou_results = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_single_file_iou, fn)
        for fn in os.listdir(mask_png_folder)
        if fn.endswith(".png")
    ]
    iou_results = [f.result() for f in futures if f.result() is not None]

# ---------------- Speichern & Visualisieren ----------------
if iou_results:
    # CSV speichern
    csv_path = "MedSAM_Spleen_iou_results.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Maskenname", "IoU"])
        writer.writerows(iou_results)
    logging.info(f"CSV gespeichert: {csv_path}")

    # Histogramm
    iou_vals = [r[1] for r in iou_results]
    avg_iou = float(np.mean(iou_vals)) if iou_vals else None

    plt.figure(figsize=(8, 6))
    plt.hist(iou_vals, bins=20, color='blue', alpha=0.7, edgecolor='black')
    plt.title('Distribution of Intersection over Union (IoU)')
    plt.xlabel('IoU')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    if avg_iou is not None:
        plt.axvline(avg_iou, color='red', linestyle='dashed', linewidth=1.5, label=f'Avg IoU: {avg_iou:.4f}')
        plt.legend()

    plot_path = os.path.join("data", "Spleen", "iou_histogram_MedSAM_Spleen.png")
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    logging.info(f"IoU-Histogramm gespeichert: {plot_path}")
else:
    logging.info("Keine IoU-Werte berechnet!")


In [ ]:
import os
import numpy as np
import cv2
import logging
import csv
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from scipy import ndimage as ndi
import nibabel as nib

# ---------------- Logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------- Pfade ----------------
mask_png_folder = os.path.join("data", "Spleen", "output_masks_updated")
segmentation_results_folder = os.path.join("data", "Spleen", "segmentation_results")
nii_root = os.path.join("data", "Spleen", "output_images")  # <— hier liegen deine .nii.gz

# ---------------- Parameter ----------------
tolerance_mm = 1.0  # Surface-Dice Toleranz in Millimetern (z.B. 1.0 oder 2.0)

# ---------------- NIfTI Spacing Cache ----------------
spacing_cache = {}  # base_name -> (dy, dx)

def get_spacing_dy_dx(base_name: str):
    """Hole (dy, dx) in mm aus <nii_root>/<base_name>.nii(.gz); fallback (1,1) mit Warnung."""
    if base_name in spacing_cache:
        return spacing_cache[base_name]
    for ext in (".nii.gz", ".nii"):
        p = os.path.join(nii_root, base_name + ext)
        if os.path.exists(p):
            nii = nib.load(p)
            zooms = nii.header.get_zooms()
            # NIfTI: (dx, dy, dz, ...) → wir brauchen (dy, dx) für EDT
            dx, dy = float(zooms[0]), float(zooms[1])
            spacing_cache[base_name] = (dy, dx)
            return spacing_cache[base_name]
    logging.warning(f"Kein NIfTI für {base_name} gefunden – spacing=(1.0, 1.0) als Fallback.")
    spacing_cache[base_name] = (1.0, 1.0)
    return spacing_cache[base_name]

# ---------------- Surface-Dice Hilfsfunktionen ----------------
def _bin2d(a):
    if a.ndim != 2:
        raise ValueError("Erwarte 2D-Maske.")
    return (a > 0).astype(np.uint8)

def _surface(mask):
    # Randpixel: Maske & ~Erosion
    if mask.sum() == 0:
        return mask.astype(bool)
    eroded = ndi.binary_erosion(mask, iterations=1, border_value=0)
    return mask.astype(bool) & (~eroded)

def surface_dice_2d(pred, gt, spacing, tol_mm: float):
    """
    Surface Dice in [0,1]: Anteil der Oberflächenpunkte beider Masken,
    deren Abstand zur jeweils anderen Oberfläche <= tol_mm ist.
    """
    p = _bin2d(pred); g = _bin2d(gt)
    ps = _surface(p); gs = _surface(g)

    # Sonderfälle
    if p.sum() == 0 and g.sum() == 0:
        return 1.0
    if ps.sum() == 0 and gs.sum() == 0:
        return 1.0
    if ps.sum() == 0 or gs.sum() == 0:
        return 0.0

    # Distanzkarten (in mm dank spacing)
    d_to_g = ndi.distance_transform_edt(~g.astype(bool), sampling=spacing)
    d_to_p = ndi.distance_transform_edt(~p.astype(bool), sampling=spacing)

    p_ok = (d_to_g[ps] <= tol_mm).sum()
    g_ok = (d_to_p[gs] <= tol_mm).sum()
    return (p_ok + g_ok) / (ps.sum() + gs.sum())

# ---------------- Einzeldatei-Verarbeitung ----------------
def process_single_file_surfacedice(mask_filename: str):
    """
    Erwartet Masken:    <base>_slice_{N}_mask.png
    Sucht Segmentierung: <base>.nii.gz_slice_{N-1}_segmentation.png  (dein bestehendes Schema)
    """
    base_name = mask_filename.split("_slice_")[0]              # z.B. spleen_63
    slice_number = mask_filename.split("_slice_")[-1].split("_")[0]
    slice_number_converted = int(slice_number) - 1
    segmentation_filename = f"{base_name}.nii.gz_slice_{slice_number_converted}_segmentation.png"

    mask_path = os.path.join(mask_png_folder, mask_filename)
    seg_path  = os.path.join(segmentation_results_folder, segmentation_filename)

    if not os.path.exists(seg_path):
        logging.warning(f"Segmentationsdatei für {mask_filename} nicht gefunden!")
        return None

    # Spacing (dy, dx) holen
    spacing = get_spacing_dy_dx(base_name)

    # Bilder laden
    mask_png = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    seg_png  = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
    if mask_png is None or seg_png is None:
        logging.error(f"Fehler beim Laden: {mask_filename}")
        return None

    # Shapes angleichen (falls nötig)
    if mask_png.shape != seg_png.shape:
        logging.warning(f"Resize {base_name} slice {slice_number}: {seg_png.shape} -> {mask_png.shape}")
        seg_png = cv2.resize(seg_png, (mask_png.shape[1], mask_png.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Binär
    m = (mask_png > 0).astype(np.uint8)
    s = (seg_png  > 0).astype(np.uint8)

    sd = surface_dice_2d(s, m, spacing=spacing, tol_mm=tolerance_mm)
    logging.info(f"{mask_filename}: Surface Dice ({tolerance_mm} mm) = {sd:.4f} (spacing={spacing})")
    return mask_filename, float(sd)

# ---------------- Parallelisieren & Sammeln ----------------
sd_results = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_single_file_surfacedice, fn)
        for fn in os.listdir(mask_png_folder)
        if fn.endswith(".png")
    ]
    sd_results = [f.result() for f in futures if f.result() is not None]

# ---------------- Speichern & Visualisieren ----------------
if sd_results:
    csv_name = f"MedSAM_Spleen_surface_dice_{str(tolerance_mm).replace('.','_')}mm_results.csv"
    with open(csv_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Maskenname", f"SurfaceDice_{tolerance_mm}mm"])
        writer.writerows(sd_results)
    logging.info(f"CSV gespeichert: {csv_name}")

    sd_vals = [r[1] for r in sd_results]
    avg_sd = float(np.mean(sd_vals)) if sd_vals else None

    plt.figure(figsize=(8, 6))
    plt.hist(sd_vals, bins=20, color='blue', alpha=0.7, edgecolor='black')
    plt.title(f'Distribution of Surface Dice ({tolerance_mm} mm)')
    plt.xlabel('Surface Dice')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    if avg_sd is not None:
        plt.axvline(avg_sd, color='red', linestyle='dashed', linewidth=1.5,
                    label=f'Avg SD ({tolerance_mm} mm): {avg_sd:.4f}')
        plt.legend()

    plot_path = os.path.join("data", "Spleen", f"surface_dice_histogram_{str(tolerance_mm).replace('.','_')}mm_MedSAM_Spleen.png")
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    logging.info(f"Surface-Dice-Histogramm gespeichert: {plot_path}")
else:
    logging.info("Keine Surface-Dice-Werte berechnet!")


In [ ]:
# --- Save results (CSV) & standardized histogram (Spleen – MedSAM) ---
import os, csv, numpy as np
import matplotlib.pyplot as plt

if dice_scores:
    # CSV (konsistent benannt)
    out_csv = r"results\Spleen\dice_MedSAM_Spleen.csv"
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Filename", "Dice"])
        writer.writerows(dice_scores)

    # Werte + Mittelwert
    dice_values = np.array([score for _, score in dice_scores], dtype=float)
    mean_val = float(np.mean(dice_values))
    print(f"Mean Dice (Spleen, MedSAM): {mean_val:.4f}")

    # Einheitliches Histogramm
    bins = np.linspace(0, 1, 21)
    plt.figure(figsize=(8, 6))
    plt.hist(dice_values, bins=bins, edgecolor="black")
    plt.xlim(0, 1); plt.ylim(bottom=0)
    plt.xlabel("Dice Score"); plt.ylabel("Frequency")
    plt.axvline(mean_val, ls="--", lw=1.5, color="red", label=f"Mean: {mean_val:.4f}")
    plt.legend(loc="upper left", framealpha=0.9)
    plt.title("Distribution of Dice Scores (Spleen, MedSAM)")
    plt.tight_layout()

    # Einheitlicher Speicherort/Name für LaTeX
    out_png = r"PICs\dice_score_histogram_MedSAM_Spleen.png"
    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    plt.savefig(out_png, dpi=200, bbox_inches="tight")
    plt.close()
else:
    logging.info("Keine Dice Scores berechnet!")


In [ ]:
'''
import os
import cv2
import matplotlib.pyplot as plt

# Verzeichnisse
mask_png_folder = os.path.join("data", "Lung Tumors", "output_2_masks_updated")
segmentation_results_folder = os.path.join("data", "Lung Tumors", "segmentation_2_results")

# Funktion zur Anzeige von drei Bildern
def show_three_images(mask_filename, slice_number):
    """
    Zeigt die Originalmaske, die erstellte Maske und die Segmentierungsvisualisierung.
    :param mask_filename: Name der Originalmaske im Ordner `output_2_masks_updated`.
    :param slice_number: Nummer des Slices für die Zuordnung von Segmentierung und Visualisierung.
    """
    # Anpassung des Dateinamens
    base_name = mask_filename.split("_slice_")[0]
    segmentation_mask_filename = f"{base_name}.nii.gz_slice_{slice_number}_segmentation.png"
    segmentation_visual_filename = f"{base_name}.nii.gz_slice_{slice_number}_visualization.png"

    # Pfade zu den Dateien
    mask_png_path = os.path.join(mask_png_folder, mask_filename)
    segmentation_mask_path = os.path.join(segmentation_results_folder, segmentation_mask_filename)
    segmentation_visual_path = os.path.join(segmentation_results_folder, segmentation_visual_filename)

    # Prüfen, ob die Dateien existieren
    if not os.path.exists(mask_png_path):
        print(f"Die Originalmaske {mask_filename} wurde nicht gefunden!")
        return
    if not os.path.exists(segmentation_mask_path):
        print(f"Die erstellte Maske {segmentation_mask_filename} wurde nicht gefunden!")
        return
    if not os.path.exists(segmentation_visual_path):
        print(f"Die Segmentierungsvisualisierung {segmentation_visual_filename} wurde nicht gefunden!")
        return

    # Dateien laden
    original_mask = cv2.imread(mask_png_path, cv2.IMREAD_GRAYSCALE)
    segmentation_mask = cv2.imread(segmentation_mask_path, cv2.IMREAD_GRAYSCALE)
    segmentation_visual = cv2.imread(segmentation_visual_path)

    # Darstellung
    plt.figure(figsize=(18, 6))

    # Originalmaske
    plt.subplot(1, 3, 1)
    plt.imshow(original_mask, cmap="gray")
    plt.title("Originalmaske")
    plt.axis("off")

    # Generierte Maske
    plt.subplot(1, 3, 2)
    plt.imshow(segmentation_mask, cmap="gray")
    plt.title("Erstellte Maske")
    plt.axis("off")

    # Segmentierungsvisualisierung
    plt.subplot(1, 3, 3)
    plt.imshow(segmentation_visual)
    plt.title("Segmentierungsvisualisierung")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Beispielaufruf
# Geben Sie die Originalmaske und die gewünschte Slice-Nummer ein
mask_filename = "lung_001_slice_01_segmentation.png"  # Beispiel
slice_number = 0  # Beispiel-Slice-Nummer (0-basiert)
show_three_images(mask_filename, slice_number)
''' 